In [10]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# List of tickers
tickers = {
    'Walmart': 'WMT',
    'McDonalds': 'MCD',
    'SP500': '^GSPC',  # S&P 500 for comparison
    'Gold': 'GC=F'
}



In [11]:

# Function to download historical data
def download_data(tickers, start_date, end_date):
    data = {}
    for name, ticker in tickers.items():
        data[name] = yf.download(ticker, start=start_date, end=end_date)['Adj Close']
    return pd.DataFrame(data)

# Download data for the specified periods
data_2006_2009 = download_data(tickers, '2006-01-01', '2009-12-31')
data_1992_1994 = download_data(tickers, '1992-01-01', '1994-12-31')

# Add a constant risk-free rate column (8% per annum, compounded daily)
risk_free_rate_daily = (1 + 0.08) ** (1/252) - 1
data_2006_2009['Bonds'] = (1 + risk_free_rate_daily) ** np.arange(len(data_2006_2009.index))
data_1992_1994['Bonds'] = (1 + risk_free_rate_daily) ** np.arange(len(data_1992_1994.index))

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['GC=F']: YFChartError("%ticker%: Data doesn't exist for startDate = 694242000, endDate = 788850000")


In [12]:
# Function to calculate portfolio returns
def portfolio_returns(weights, returns):
    return np.dot(weights, returns.mean()) * 252

# Function to calculate portfolio volatility
def portfolio_volatility(weights, returns):
    return np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))

# Function to calculate Sharpe ratio
def sharpe_ratio(weights, returns, risk_free_rate=risk_free_rate_daily):
    return (portfolio_returns(weights, returns) - risk_free_rate * 252) / portfolio_volatility(weights, returns)

# Function to optimize portfolio
def optimize_portfolio(returns):
    num_assets = len(returns.columns)
    args = (returns,)
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})
    bounds = tuple((0, 1) for asset in range(num_assets))
    result = minimize(lambda weights: -sharpe_ratio(weights, *args),
                      num_assets * [1. / num_assets,],
                      method='SLSQP',
                      bounds=bounds,
                      constraints=constraints)
    return result

In [7]:
# Calculate returns
returns_2006_2009 = data_2006_2009.pct_change().dropna()
returns_1992_1994 = data_1992_1994.pct_change().dropna()

# Optimize portfolio for both periods
optimized_portfolio_2006_2009 = optimize_portfolio(returns_2006_2009)
optimized_portfolio_1992_1994 = optimize_portfolio(returns_1992_1994)

# Extract the optimized weights
weights_2006_2009 = optimized_portfolio_2006_2009.x
weights_1992_1994 = optimized_portfolio_1992_1994.x

# Display results
print("Optimized portfolio weights (2006-2009):", weights_2006_2009)
print("Optimized portfolio weights (1992-1994):", weights_1992_1994)

# Calculate average weights
average_weights = (weights_2006_2009 + weights_1992_1994) / 2
portfolio_allocation = pd.DataFrame(average_weights, index=returns_2006_2009.columns, columns=['Allocation'])

# Display final portfolio allocation
print("Final portfolio allocation:")
print(portfolio_allocation)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['GC=F']: YFChartError("%ticker%: Data doesn't exist for startDate = 694242000, endDate = 788850000")


Optimized portfolio weights (2006-2009): [9.01720214e-18 5.02990809e-01 1.31405409e-16 4.97009191e-01
 5.79725545e-17]
Optimized portfolio weights (1992-1994): [0.2 0.2 0.2 0.2 0.2]
Final portfolio allocation:
           Allocation
Walmart      0.100000
McDonalds    0.351495
SP500        0.100000
Gold         0.348505
Bonds        0.100000


<ipython-input-7-ecf9fd60f75b>:56: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns_2006_2009 = data_2006_2009.pct_change().dropna()
<ipython-input-7-ecf9fd60f75b>:57: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns_1992_1994 = data_1992_1994.pct_change().dropna()
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.10/dist-

In [13]:
# Function to calculate portfolio value over time
def calculate_portfolio_value(initial_investment, weights, returns):
    portfolio_values = (returns + 1).cumprod().dot(weights) * initial_investment
    return portfolio_values

# Calculate portfolio values for $100,000 initial investment
initial_investment = 100000
portfolio_value_2006_2009 = calculate_portfolio_value(initial_investment, weights_2006_2009, returns_2006_2009)
portfolio_value_1992_1994 = calculate_portfolio_value(initial_investment, weights_1992_1994, returns_1992_1994)

# Display portfolio values over time
print("Portfolio value over time (2006-2009):")
print(portfolio_value_2006_2009)

print("Portfolio value over time (1992-1994):")
print(portfolio_value_1992_1994)

Portfolio value over time (2006-2009):
Date
2006-01-04    100749.866311
2006-01-05    100098.111523
2006-01-06    101653.160784
2006-01-09    103508.839027
2006-01-10    103104.341860
                  ...      
2009-12-23    208850.339895
2009-12-24    209962.541997
2009-12-28    210572.133672
2009-12-29    209583.288417
2009-12-30    207891.995173
Length: 1007, dtype: float64
Portfolio value over time (1992-1994):
Series([], dtype: float64)
